<a href="https://colab.research.google.com/github/trahayan/btl/blob/main/CNN_eye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive/")
#import tensorflow as tf

Mounted at /content/drive/


In [ ]:
cd /content

/content


In [ ]:
!unzip Eye_diseases5.zip

Archive:  Eye_diseases5.zip
   creating: Eye_diseases/
   creating: Eye_diseases/Bulging_Eyes/
  inflating: Eye_diseases/Bulging_Eyes/1313.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/1515.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/22.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/33.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/55.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-10.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-11.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-12.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-13.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-2.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-3.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-4.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-5.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-6.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-7.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image-9.jpeg  
  inflating: Eye_diseases/Bulging_Eyes/image.jpeg  
  infl

In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

# Define paths and constants
PATH = "./drive/MyDrive/Colab Notebooks/Data/Eye_diseases"
data_dir_list = os.listdir(PATH)

In [4]:
# Load and preprocess data
img_rows = 224
img_cols = 224
num_channel = 3
img_data_list = []
classes_names_list = []
target_column = []

for dataset in data_dir_list:
    classes_names_list.append(dataset)
    print("Loading data from {} folder".format(dataset))
    img_list = os.listdir(PATH + "/" + dataset)
    for img in img_list:
        input_img = cv2.imread(PATH + "/" + dataset + "/" + img)
        input_img_resize = cv2.resize(input_img, (img_rows, img_cols))
        img_data_list.append(input_img_resize)
        target_column.append(dataset)

num_classes = len(classes_names_list)
print("num_classes",num_classes)
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print("Shape of image data",img_data.shape)
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

# Encode labels
Labelencoder = LabelEncoder()
target_column = Labelencoder.fit_transform(target_column)
target_column_hotcoded = to_categorical(target_column, num_classes)



Loading data from Bulging_Eyes folder
Loading data from Cataracts folder
Loading data from Crossed_Eyes folder
Loading data from Glaucoma folder
Loading data from Uveitis folder
num_classes 5
Shape of image data (270, 224, 224, 3)


In [5]:
# Shuffle and split data
X, Y = shuffle(img_data, target_column_hotcoded, random_state=2)
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

train_generator = datagen.flow(X_train, y_train, batch_size=32)


In [7]:
# Load VGG16 pretrained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, num_channel))


for layer in base_model.layers[:-3]:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          65,6

 Total params: 14,780,997 (56.39 MB)

 Trainable params: 4,785,925 (18.26 MB)

 Non-trainable params: 9,995,072 (38.13 MB)

In [ ]:
# Train model VGG có tăng cường DL
num_epoch = 50
batch_size = 32
steps_per_epoch = max(len(X_train) // batch_size, 1)
early_stop = EarlyStopping(
    monitor='val_loss',  # Theo dõi tổn thất trên tập kiểm định
    patience=5,          # Dừng nếu val_loss không cải thiện sau 5 epoch
    restore_best_weights=True  # Khôi phục trọng số tốt nhất
)
history1 = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epoch,
    validation_data=(X_val, y_val),
    verbose=1,
    callbacks=[early_stop],
)

# Evaluate model
score = model.evaluate(X_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.3295 - loss: 2.3831 - val_accuracy: 0.5366 - val_loss: 1.5886
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.1250 - loss: 2.2607 - val_accuracy: 0.5366 - val_loss: 1.4454
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 226ms/step - accuracy: 0.4206 - loss: 1.4799 - val_accuracy: 0.6341 - val_loss: 1.2899
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.4062 - loss: 1.2839 - val_accuracy: 0.5122 - val_loss: 1.2014
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 256ms/step - accuracy: 0.4563 - loss: 1.3553 - val_accuracy: 0.6098 - val_loss: 1.0643
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5000 - loss: 1.0887 - val_accuracy: 0.6341 - val_loss: 1.1593
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 223ms/step - accuracy: 0.5619 - loss: 1.2272 - val_accuracy: 0.6341 - val_loss: 0.9524
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5938 - loss: 0.9321 - val_accuracy: 0.5854 - val_loss: 1.1351
Epoch 9/5

In [ ]:
#Train VGG ko tăng cường dữ liệu
num_epoch = 50
batch_size = 32
steps_per_epoch = max(len(X_train) // batch_size, 1)
early_stop = EarlyStopping(
    monitor='val_loss',  # Theo dõi tổn thất trên tập kiểm định
    patience=5,          # Dừng nếu val_loss không cải thiện sau 5 epoch
    restore_best_weights=True  # Khôi phục trọng số tốt nhất
)
history=model.fit(X_train,y_train,batch_size=batch_size,epochs=num_epoch,verbose=1,validation_data=(X_val,y_val), callbacks=[early_stop])
score = model.evaluate(X_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 226ms/step - accuracy: 0.9760 - loss: 0.0994 - val_accuracy: 0.7805 - val_loss: 0.6652
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 225ms/step - accuracy: 0.9928 - loss: 0.0679 - val_accuracy: 0.7561 - val_loss: 0.7798
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 207ms/step - accuracy: 0.9950 - loss: 0.0444 - val_accuracy: 0.8293 - val_loss: 0.7408
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - accuracy: 1.0000 - loss: 0.0272 - val_accuracy: 0.8293 - val_loss: 0.6062
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 207ms/step - accuracy: 1.0000 - loss: 0.0146 - val_accuracy: 0.7805 - val_loss: 0.8448
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - accuracy: 1.0000 - loss: 0.0085 - val_accuracy: 0.8049 - val_loss: 0.8501
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 232ms/step - accuracy: 1.0000 - loss: 0.0043 - val_accuracy: 0.7805 - val_loss: 1.1207
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 233ms/step - accuracy: 1.0000 - loss: 0.0040 - val_accuracy: 0.8537 - val_loss:

AttributeError: 'History' object has no attribute 'evaluate'

In [9]:
# Build model
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_rows, img_cols, num_channel)))
model1.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(0.3))

model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(0.3))

model1.add(Flatten())
model1.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01)))

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │      47,776,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 47,796,677 (182.33 MB)

 Trainable params: 47,796,677 (182.33 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Train model ko tăng cường dữ liệu
num_epoch = 50
batch_size = 32
early_stop = EarlyStopping(
    monitor='val_loss',  # Theo dõi tổn thất trên tập kiểm định
    patience=5,          # Dừng nếu val_loss không cải thiện sau 5 epoch
    restore_best_weights=True  # Khôi phục trọng số tốt nhất
)
hist1 = model1.fit(
    X_train,y_train,batch_size=batch_size,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=num_epoch,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)

# Evaluate model
score = model1.evaluate(X_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 5s/step - accuracy: 0.2393 - loss: 20.3506 - val_accuracy: 0.4146 - val_loss: 7.0045
Epoch 2/50
1/5 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.3793 - loss: 12.6422

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 807ms/step - accuracy: 0.3793 - loss: 12.6422 - val_accuracy: 0.2195 - val_loss: 6.9778
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 35s 5s/step - accuracy: 0.2457 - loss: 9.5385 - val_accuracy: 0.4146 - val_loss: 7.6786
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 396ms/step - accuracy: 0.3793 - loss: 7.6814 - val_accuracy: 0.4146 - val_loss: 7.7162
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 24s 5s/step - accuracy: 0.3562 - loss: 7.6209 - val_accuracy: 0.4146 - val_loss: 7.4823
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 399ms/step - accuracy: 0.4828 - loss: 7.3563 - val_accuracy: 0.4146 - val_loss: 7.3794
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 26s 6s/step - accuracy: 0.3856 - loss: 7.1237 - val_accuracy: 0.4146 - val_loss: 6.7152
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 416ms/step - accuracy: 0.4828 - loss: 6.6589 - val_accuracy: 0.4146 - val_loss: 6.5968
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.4545 - loss: 6.3726 - val_accuracy: 0.4390 - val_loss: 6.0315
Epoch 10/5

In [12]:
# Train model tăng cường dữ liệu
num_epoch = 50
batch_size = 32

hist2 = model1.fit(
    train_generator,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=num_epoch,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)

# Evaluate model
score = model1.evaluate(X_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 29s 5s/step - accuracy: 0.4996 - loss: 2.1870 - val_accuracy: 0.5122 - val_loss: 1.9161
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 703ms/step - accuracy: 0.4688 - loss: 2.3070 - val_accuracy: 0.5122 - val_loss: 1.8931
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 34s 5s/step - accuracy: 0.3768 - loss: 2.1568 - val_accuracy: 0.6098 - val_loss: 2.0821
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 397ms/step - accuracy: 0.3125 - loss: 2.2917 - val_accuracy: 0.6098 - val_loss: 2.0937
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 34s 5s/step - accuracy: 0.5027 - loss: 2.0637 - val_accuracy: 0.5610 - val_loss: 2.0323
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 326ms/step - accuracy: 0.5646 - loss: 1.9347
Test loss: 1.9696108102798462
Test accuracy: 0.550000011920929
